# Building a Recommendation System in PySpark - Lab

## Introduction

In this lab, we will implement a movie recommendation system using ALS in Spark programming environment. Spark's machine learning library `ml` comes packaged with a very efficient implementation of the ALS algorithm that we looked at in the previous lesson. The lab will require you to put into practice your Spark programming skills for creating and manipulating PySpark DataFrames. We will go through a step-by-step process into developing a movie recommendation system using ALS and PySpark using the `MovieLens` dataset that we used in a previous lab.

Note: You are advised to refer to [PySpark documentation](http://spark.apache.org/docs/2.2.0/api/python/index.html) heavily for completing this lab as it will introduce a few new methods. 


## Objectives

In this lab you will: 

- Use Spark to train and cross-validate an ALS model 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 

## Building a Recommendation System

We have seen how recommendation systems have played an integral part in the success of Amazon (books, items), Pandora/Spotify (music), Google (news, search), YouTube (videos) etc.  For Amazon, these systems bring more than 30% of their total revenue. For Netflix, 75% of movies that people watch are based on some sort of recommendation.

> The goal of recommendation systems is to find what is likely to be of interest to the user. This enables organizations to offer a high level of personalization and customer tailored services.


For online video content services like Netflix and Hulu, the need to build robust movie recommendation systems is extremely important. An example of a recommendation system is such as this:

1.    User A watches Game of Thrones and Breaking Bad 
2.    User B performs a search query for Game of Thrones 
3.    The system suggests Breaking Bad to user B from data collected about user A 


This lab will guide you through a step-by-step process into developing such a movie recommendation system. We will use the `MovieLens` dataset to build a movie recommendation system using the collaborative filtering technique with Spark's Alternating Least Squares implementation. After building that recommendation system, we will go through the process of adding a new user to the dataset with some new ratings and obtaining new recommendations for that user.

### Importing the Data

* Initialize a `SparkSession` object
* Import the dataset found at `'./data/ratings.csv'` into a PySpark DataFrame

In [1]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Initialize a SparkSession object
spark = SparkSession.builder.appName("MovieRecommendations").getOrCreate()

# Read in the dataset into a PySpark DataFrame
ratings_path = "./data/ratings.csv"
movie_ratings = spark.read.csv(ratings_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame to verify the data import
movie_ratings.show()



+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



Check the data types of each of the columns to ensure that they are a type that makes sense given the column.

In [2]:
# Check the data types of each column
movie_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



We aren't going to need the timestamp, so we can go ahead and remove that column.

In [3]:
movie_ratings = movie_ratings.drop("timestamp")
movie_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



### Fitting the Alternating Least Squares Model

Because this dataset is already preprocessed for us, we can go ahead and fit the Alternating Least Squares model.

* Import `ALS` from `pyspark.ml.recommendation` module 
* Use the `.randomSplit()` method on the pyspark DataFrame to separate the dataset into training and test sets
* Fit the Alternating Least Squares Model to the training dataset. Make sure to set the `userCol`, `itemCol`, and `ratingCol` to the appropriate columns given this dataset. Then fit the data to the training set and assign it to a variable model  

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Splitting the dataset into training and testing sets
train_data, test_data = movie_ratings.randomSplit([0.8, 0.2], seed=42)

# Building the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(
    maxIter=10,
    regParam=0.01,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

# Fit the ALS model to the training set
model = als.fit(train_data)

Now you've fit the model, and it's time to evaluate it to determine just how well it performed.

* Import `RegressionEvalutor` from `pyspark.ml.evaluation` 
* Generate predictions with your model for the test set by using the `.transform()` method on your ALS model 
* Evaluate your model and print out the RMSE from your test set 

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator

# Generate predictions with your model for the test set
predictions = model.transform(test_data)

# Evaluate the model by computing the RMSE on the test data
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 1.104314805450727


### Cross-validation to Find the Optimal Model

Let's now find the optimal values for the parameters of the ALS model. Use the built-in `CrossValidator` in PySpark with a suitable param grid and determine the optimal model. Try with the parameters:

* regularization = [0.01, 0.001, 0.1]
* rank = [4, 10, 50]


In [6]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Initialize the ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

# Create the parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(als.regParam, [0.01, 0.001, 0.1]) \
    .addGrid(als.rank, [4, 10, 50]) \
    .build()

# Instantiate the CrossValidator estimator
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)  # You can adjust the number of folds as needed

# Fit the CrossValidator to the training data
cvModel = crossval.fit(train_data)

# Get the best ALS model from cross-validation
best_model = cvModel.bestModel

# Print the best rank and regularization parameter
print(f"Best rank: {best_model.rank}")
print(f"Best regularization parameter: {best_model._java_obj.parent().getRegParam()}")


Best rank: 4
Best regularization parameter: 0.1


### Incorporating the names of the movies

When we make recommendations, it would be ideal if we could have the actual name of the movie used rather than just an ID. There is another file called `'./data/movies.csv'` that contains all of the names of the movies matched up to the `movie_id` that we have in the ratings dataset.

* Import the data into a Spark DataFrame 
* Look at the first 5 rows

In [7]:
# Import the movie titles data into a Spark DataFrame
movies_path = "./data/movies.csv"
movie_titles = spark.read.csv(movies_path, header=True, inferSchema=True)

# Look at the first 5 rows
movie_titles.show(5)


+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



We will eventually be matching up the movie ids with the movie titles. In the cell below, create a function `name_retriever()` that takes in a `movie_id` and returns a string that represents the movie title. 

> Hint: It's possible to do this operation in one line with the `df.where()` or the `df.filter()` methods 

In [8]:
def name_retriever(movie_id, movie_title_df):
    # Filter the DataFrame to find the movie title with the given movie_id
    movie = movie_title_df.filter(movie_title_df.movieId == movie_id).select("title").first()
    
    # Extract the movie title as a string
    if movie:
        return movie.title
    else:
        return "Movie not found"

In [9]:
# Test the function
print(name_retriever(1023, movie_titles))

Winnie the Pooh and the Blustery Day (1968)


## Getting Recommendations

Now it's time to actually get some recommendations! The ALS model has built-in methods called `.recommendForUserSubset()` and `.recommendForAllUsers()`. We'll start off with using a subset of users. 

In [10]:
users = movie_ratings.select(als.getUserCol()).distinct().limit(1)
userSubsetRecs = model.recommendForUserSubset(users, 10)
recs = userSubsetRecs.take(1)

We can now see we have a list of rows with recommended items. Now try and get the name of the top recommended movie by way of the function you just created, using number one item for this user.

In [11]:
# use indexing to obtain the movie id of top predicted rated item
first_recommendation = recs[0]['recommendations'][0][0]

# use the name retriever function to get the values
name_retriever(first_recommendation,movie_titles)

'High Noon (1952)'

Of course, you can also make recommendations for everyone, although this will take longer. In the next line, we are creating an RDD with the top 5 recommendations for every user and then selecting one user to find out his predictions:

In [12]:
recommendations = model.recommendForAllUsers(5)
recommendations.where(recommendations.userId == 3).collect()

[Row(userId=3, recommendations=[Row(movieId=8633, rating=7.008184432983398), Row(movieId=2517, rating=6.64518928527832), Row(movieId=2013, rating=6.470295429229736), Row(movieId=417, rating=6.38945198059082), Row(movieId=3837, rating=6.306672096252441)])]

### Getting Predictions for a New User

Now, it's time to put together all that you've learned in this section to create a function that will take in a new user and some movies they've rated and then return $n$ number of highest recommended movies. This function will have multiple different steps to it:

* Adding the new ratings into the DataFrame (hint: look into using the `.union()` method) 
* Fitting the ALS model  
* Make recommendations for the user of choice 
* Print out the names of the top $n$ recommendations in a reader-friendly manner 

The function should take in the parameters: 

* `user_id` : int 
* `new_ratings` : list of tuples in the format (user_id, item_id, rating)
* `rating_df` : spark DF containing ratings
* `movie_title_df` : spark DF containing movie titles
* `num_recs` : int

Rate new movies

```python
[Row(movieId=3253, title='Wayne's World (1992)', genres='Comedy'),
 Row(movieId=2459, title='Texas Chainsaw Massacre, The (1974)', genres='Horror'),
 Row(movieId=2513, title='Pet Sematary (1989)', genres='Horror'),
 Row(movieId=6502, title='28 Days Later (2002)', genres='Action|Horror|Sci-Fi'),
 Row(movieId=1091, title='Weekend at Bernie's (1989)', genres='Comedy'),
Row(movieId=441, title='Dazed and Confused (1993)', genres='Comedy'),
Row(movieId=370, title='Naked Gun 33 1/3: The Final Insult (1994)', genres='Action|Comedy')]

```

In [29]:
def get_new_user_recommendations(user_id, new_ratings, rating_df, movie_title_df, num_recs):
    from pyspark.sql import Row
    from pyspark.ml.recommendation import ALS
    
    # Create a new DataFrame with the new ratings
    new_ratings_df = spark.createDataFrame([Row(userId=user_id, movieId=item_id, rating=rating) for (item_id, rating) in new_ratings])
    
    # Append the new ratings to the existing ratings DataFrame
    updated_rating_df = rating_df.union(new_ratings_df)
    
    # Build and fit the ALS model with the best rank and regularization parameter
    als = ALS(
        maxIter=4,
        regParam=0.1,  # Best regularization parameter
        userCol="userId",
        itemCol="movieId",
        ratingCol="rating",
        coldStartStrategy="drop"
    )
    model = als.fit(updated_rating_df)
    
    # Get recommendations for the new user using the existing model
    user_recommendations = model.recommendForUserSubset(updated_rating_df.select("userId").distinct(), num_recs)
    
    # Extract the movie titles for the recommendations
    top_recommendations = user_recommendations.select("recommendations.movieId").collect()[0][0]
    
    # Retrieve and print the names of the top recommended movies
    recommended_movies = [name_retriever(movie_id, movie_title_df) for movie_id in top_recommendations]
    
    return recommended_movies



In [30]:
# Example usage:
user_id = 1  # Replace with the desired user ID

# List of (movieId, rating) for new ratings
new_ratings = [
    (3253, 4.0),  # Wayne's World (1992)
    (2459, 3.5),  # Texas Chainsaw Massacre, The (1974)
    (2513, 3.0),  # Pet Sematary (1989)
    (6502, 4.5),  # 28 Days Later (2002)
    (1091, 4.0),  # Weekend at Bernie's (1989)
    (441, 3.5),   # Dazed and Confused (1993)
    (370, 4.0)    # Naked Gun 33 1/3: The Final Insult (1994)
]

num_recs = 10  # Number of recommendations to generate
recommended_movies = get_new_user_recommendations(user_id, new_ratings, movie_ratings, movie_titles, num_recs)

print(f"Top {num_recs} recommended movies for user {user_id}:")
for i, movie in enumerate(recommended_movies, start=1):
    print(f"{i}. {movie}")



Py4JJavaError: An error occurred while calling o9703.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 5400.0 failed 1 times, most recent failure: Lost task 4.0 in stage 5400.0 (TID 12928) (Leo executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.s

So here we have it! Our recommendation system is generating recommendations for the top 10 movies. 




## Level up (Optional)


* Create a user interface to allow users to easily choose items and get recommendations 

* Use IMDB links to scrape user reviews from IMDB and using basic NLP techniques, create extra embeddings for ALS model  

* Create a hybrid recommender system using features like genre 

## Summary

In this lab, you built a model using Spark, performed some parameter selection, and updated the model every time new user preferences came in. You looked at how Spark's ALS implementation can be used to build a scalable and efficient recommendation system. You also saw that such systems can become computationally expensive and using them with an online system could be a problem with traditional computational platforms. Spark's distributed computing architecture provides a great solution to deploy such recommendation systems for real-world applications (think Amazon, Spotify).